In [19]:
import numpy as np

In [20]:
# List of key parameters




mumax4=0.17 #1/day
thetamax4=0.57 #molATP/cmolX/day in Unit4
kla=20 #1/day
Ks_mu4=0.002 #cmol/L
Co_sat=7/1000/32 #mol/l
Ko_mu4=Co_sat*0.15 #mol/L
Ks_theta4=Ks_mu4/1e6 #cmol/L
Ko_theta4=Ko_mu4/1e6 #mol/L
Q1=1.5  #L

In [21]:
MM = np.array([12,1,16,14,31,23])

In [22]:
#Composition specs 'C H1.854 O0.48 N0.095 P0.0036 Na0.0016'

In [23]:
#Composition specifications:
#order         C;   H;  O;  N;  P;  Na;   

S1 = np.array([1,1.854,0.48,0.095,0.0036,0.0016])
S2 = np.array([1,1.75,0.59,0.125,0.005825,0.00172])
S3 = np.array([1,1.46,0.6,0.02,0.0025,0.0023])
S4 = np.array([1,1.7,0.75,0.02,0.0275,0.00118])
S5 = np.array([1,1.8,0.5,0.08,0.02,0])
S6 = np.array([1,1.7,0.55,0.13,0.02,0])

Calculating the amount of food Ben eats and all the NE's associated with this.

In [24]:
wetFractionFood = 0.6
nonEdibleFraction = 0.6

In [49]:
kiloJoulesRequired = 9000
#                                     Prot, Fat, Carbs
foodFractionsDryMassBasis = np.array([0.36,0.21,0.43])
energyPerDryGramFoodGroup = np.array([17,38,17])
energyPerDryGramFood = np.sum(foodFractionsDryMassBasis * energyPerDryGramFoodGroup)
massChonDryFoodPerDay = kiloJoulesRequired / energyPerDryGramFood
molsChonDryFoodPerDay = massChonDryFoodPerDay / (MM[0]*S1[0] + MM[1]*S1[1] + MM[2]*S1[2] + MM[3]*S1[3])
massDryFoodPerDay = molsChonDryFoodPerDay * np.sum(MM*S1)

massWetFoodPerDay = massDryFoodPerDay / (1 - wetFractionFood)

massWaterInFoodPerDay = massWetFoodPerDay - massDryFoodPerDay

massNonEdibleDryFoodPerDay = massDryFoodPerDay / (1 - nonEdibleFraction) * nonEdibleFraction

massNonEdibleWetFoodPerDay = massNonEdibleDryFoodPerDay / (1 - wetFractionFood)

massWaterInNonEdibleFoodPerDay = massNonEdibleWetFoodPerDay - massNonEdibleDryFoodPerDay

print('Mass dry food per day =',np.round(massDryFoodPerDay,2))
print('Mass wet food per day =',np.round(massWetFoodPerDay,2))

print('\nMass dry non-edible food per day =',np.round(massNonEdibleDryFoodPerDay,2))
print('Mass wet non-edible food per day =',np.round(massNonEdibleWetFoodPerDay,2))

print('\nMass water in edible food per day =',np.round(massWaterInFoodPerDay,2))
print('Mass water in non-edible food per day =',np.round(massWaterInNonEdibleFoodPerDay,2))

Mass dry food per day = 423.09
Mass wet food per day = 1057.73

Mass dry non-edible food per day = 634.64
Mass wet non-edible food per day = 1586.6

Mass water in edible food per day = 634.64
Mass water in non-edible food per day = 951.96


In [28]:
molsDryFoodPerDay = molsChonDryFoodPerDay
molsNonEdibleDryFoodPerDay = massNonEdibleDryFoodPerDay / np.sum(MM * S3)

print('Mols dry food per day =',np.round(molsDryFoodPerDay,3))
print('Mols dry non-edible food per day =',np.round(molsNonEdibleDryFoodPerDay,3))

Mols dry food per day = 18.385
Mols dry non-edible food per day = 27.04


Solving the Hydroponics unit:

In [32]:
#                               P    Na   NH3   H2O   CO2   O2   S1      S3
hydroponicsMatrixA=np.matrix([[0,   0,   0,    0,    1,    0,   1,       1     ],  #C
                              [0,   0,   3,    2,    0,    0,   1.853 ,  1.46  ],  #H
                              [0,   0,   0,    1,    2,    2,   0.479 ,  0.6   ],  #O
                              [0,   0,   1,    0,    0,    0,   0.096 ,  0.02  ],  #N
                              [1,   0,   0,    0,    0,    0,   0.0036,  0.0025],  #P
                              [0,   1,   0,    0,    0,    0,   0.0016,  0.0023],  #Na
                              [0,   0,   0,    0,    0,    0,   1,       0     ],  #S1 fed to ben
                              [0,   0,   0,    0,    0,    0,   0,       1     ]]) #S2 non edible food

hydroponicsMatrixB=np.matrix([0, 0, 0, 0, 0, 0, molsDryFoodPerDay, molsNonEdibleDryFoodPerDay ]).T

hydroponicsRates=np.linalg.solve(hydroponicsMatrixA, hydroponicsMatrixB)

U1RateP, U1RateNa, U1RateNH3, U1RateH2O, U1RateCO2, U1RateO2, U1RateS1, U1RateS3 = hydroponicsRates.flat

print(hydroponicsRates)

[[ -0.13378747]
 [ -0.09160862]
 [ -2.30580016]
 [-33.31457732]
 [-45.42539987]
 [ 49.56738668]
 [ 18.38542319]
 [ 27.03997669]]


Modelling the Urine:

Urine can be modelled as 40g of urea exiting the body on a daily basis. Allow for a seperate P and Na stream in your unit 2 matrix and note that this should be added to the total urine stream. Ben produces 1.5 liter of urine a day. The urea in the urine will spontaneously undergo the following decomposition:

$$CH_4ON_2 +H_2O \to CO_2+2NH_3$$

Ensure to calculate a $NH_3$ concentration in the urine.

In [33]:
massUreaPerDay = 40
molarMassUrea = 12 + 4 + 16 + 28
molsUreaPerDay = massUreaPerDay / molarMassUrea

Solving Ben's metabolism:

In [34]:
#                      P    Na    H2O   CO2   O2   S1      S2     U
benMatrixA=np.matrix([[0,   0,    0,    1,    0,   1,      S2[0], 1],  #C
                      [0,   0,    2,    0,    0,   1.853 , S2[1], 4],  #H
                      [0,   0,    1,    2,    2,   0.479 , S2[2], 1],  #O
                      [0,   0,    0,    0,    0,   0.096 , S2[3], 2],  #N
                      [1,   0,    0,    0,    0,   0.0036, S2[4], 0],  #P
                      [0,   1,    0,    0,    0,   0.0016, S2[5], 0],  #Na
                      [0,   0,    0,    0,    0,   1,      0,     0],  #S1 fed to ben
                      [0,   0,    0,    0,    0,   0,      0,     1]]) #U leaving

benMatrixB = np.matrix([0, 0, 0, 0, 0, 0, -molsDryFoodPerDay, molsUreaPerDay ]).T

benRates = np.linalg.solve(benMatrixA, benMatrixB)

U2RateP, U2RateNa, U2RateH2O, U2RateCO2, U2RateO2, U2RateS1, U2RateS2, U2RateU = benRates.flat

print(benRates)

[[  0.04607183]
 [  0.02347694]
 [ 12.6790902 ]
 [ 14.26541818]
 [-17.55372257]
 [-18.38542319]
 [  3.45333834]
 [  0.66666667]]


Anaerobic digester

In [39]:
#Start by calculating the mass of humanure:
waterFractionHumanure = 0.75
molarMassHumanure = np.sum(MM * S2)
molsDryHumanurePerDay = U2RateS2
massDryHumanurePerDay = molsDryHumanurePerDay * molarMassHumanure
print('Mass dry humanure =',massDryHumanurePerDay)

massWaterInHumanure = massDryHumanurePerDay / (1 - waterFractionHumanure) * waterFractionHumanure
print(massWaterInHumanure)

Mass dry humanure = 86.88645885282592
260.6593765584778


In [61]:
#Calculate the mass of Substrate and thus the required amount of water:
substrateMolsPerElement = molsNonEdibleDryFoodPerDay * S3 + U2RateS2 * S2
print('Substrate mols per element fed to Unit 3:',np.round(substrateMolsPerElement,2))
U3S = substrateMolsPerElement / substrateMolsPerElement[0]      #This is the cmol formula for the AD substrate
print('\nAD Substrate fed to unit 3 cmol formula:',np.round(U3S,4))

massADSubstrate = (molsNonEdibleDryFoodPerDay + U2RateS2) * np.sum(U3S * MM)
print('\nMass AD Substrate =',massADSubstrate)

Substrate mols per element fed to Unit 3: [30.49 45.52 18.26  0.97  0.09  0.07]

AD Substrate fed to unit 3 cmol formula: [1.     1.4928 0.5989 0.0319 0.0029 0.0022]

Mass AD Substrate = 721.5255276646437


In [67]:
# Now calculate total water coming in via S2 and S3:
totalWaterS2AndS3 = (massWaterInHumanure + massWaterInNonEdibleFoodPerDay) / 1000  #divide by 1000 to get L
requiredADSubstrateConcentration = 130 #g/L
requiredADTotalWater = massADSubstrate / requiredADSubstrateConcentration  #L
waterAddedToADPerDayNoRecycle = requiredADTotalWater - totalWaterS2AndS3
print(rorequiredADTotalWater)
print('The total water that needs to be added to the AD per day before considering the recycle is :', waterAddedToADPerDayNoRecycle)

5.550196366651106
The total water that needs to be added to the AD per day before considering the recycle is : 4.337578386874901


In [36]:
#Key parameters

thetamax3=0.55 # Unit 3
mumax3=0.07 #1/day
Ks_mu3=0.04 #cmol/L
Ks_theta3=0.008/1e6  #cmol/L
Na_conc=300/1000/23 #mol/L (300mg/L in clean water)